In [1]:
# import das bibliotecas
import numpy as np
import gurobipy as gb
from gurobipy import GRB

Considere o seguinte problema de programação inteira

\begin{align*}
\max \ & 4 x_1 - 2 * x_2 \\
& 7 x_1 - x_2 \leq 14 \\
& x_2 \leq 3 \\
& 2 x_1 - 2 x_2 \leq 3 \\
& x_1, x_2 \in \mathbb{Z}_{+}
\end{align*}

In [2]:
# modelo
mip = gb.Model("mip")

# variáveis
x1 = mip.addVar(vtype=GRB.INTEGER, name="x1")
x2 = mip.addVar(vtype=GRB.INTEGER, name="x2")

# função objetivo
mip.setObjective(4 * x1 - x2, GRB.MAXIMIZE)

# restrições
mip.addConstr(7 * x1 - 2 * x2 <= 14, name='restricao1')
mip.addConstr(x2 <= 3, name='restricao2')
mip.addConstr(2 * x1 - 2 * x2 <= 3, name='restricao3')

# escrevendo .LP
mip.write("mip.lp")

# configurando parametros
mip.Params.timeLimit = 60
mip.Params.MIPGap = 0.0001
#mip.params.outputflag = 0 # desabilitando o log

# resolvendo o problema
mip.optimize()

# imprimindo solução
if mip.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % mip.objVal)
    for v in mip.getVars():
        print('%s = %g' % (v.varName, v.x))
    print('Otimização finalizou com status %d' % mip.status)
elif mip.status == GRB.INFEASIBLE:
    print('Modelo é inviável')
elif mip.status == GRB.UNBOUNDED:
    print('Modelo é ilimitado')
else:
    print('Otimização finalizou com status %d' % mip.status)

Using license file /home/jossian/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter timeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Parameter MIPGap unchanged
   Value: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x2fecc6d6
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Found heuristic solution: objective 4.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, ga

### Considere a relaxação linear do MIP, a qual denominamos por $S$. 

\begin{align*}
\min \ & 4 x_1 - x_2 \\
& 7 x_1 - x_2 \leq 14 \\
& x_2 \leq 3 \\
& 2 x_1 - 2 x_2 \leq 3 \\
& x_1, x_2 \in \mathbb{R}_{+}
\end{align*}

## Crie uma lista $L$ e coloque $S$ nesta lista: $L = \{ S \}$.

## Retire $S$ de $L$ e resolva $S$.

In [5]:
# modelo
S = gb.Model("S")

# variáveis
x1 = S.addVar(name="x1")
x2 = S.addVar(name="x2")

# função objetivo
S.setObjective(4 * x1 - x2, GRB.MAXIMIZE)

# restrições
S.addConstr(7 * x1 - 2 * x2 <= 14)
S.addConstr(x2 <= 3)
S.addConstr(2 * x1 - 2 * x2 <= 3)

# configurando parametros
S.Params.timeLimit = 60

# resolvendo o modelo
S.optimize()

# imprimindo as solução
if S.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % np.round(S.objVal,1))
    for v in S.getVars():
        print('%s = %g' % (v.varName, np.round(v.x,1)))
elif S.status == GRB.INF_OR_UNBD:
    print('Modelo é inviável ou ilimitado')
else:
    print('Otimização finalizou com status %d' % S.status)

Changed value of parameter timeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xa0c41c97
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4305000e+00   1.350000e-02   0.000000e+00      0s
       1    8.4285714e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  8.428571429e+00
Valor ótimo: 8.4
x1 = 2.9
x2 = 3


## A solução de $S$ é $\overline{x} = (2.9, 3)$ com $\overline{z} = 8.4$.

## Defina $\underline{z} = -\infty$.

## A variável $x_1$ não é inteira.

## Crie dois novos problemas $S_1$ e $S_2$, onde $S_1 = S \cup \{ x_1 \leq 2 \}$ e 
$S_2 = S \cup \{ x_1 \geq 3 \}$.

## Adicione $S_1$ e $S_2$ a lista de vértices ativos:
$L = \{ S_1, S_2 \}$.

## Retire $S_1$ de $L$ e resolva $S_1$,  $L = \{ S_2 \}$.

In [6]:
# modelo
S1 = gb.Model("S1")

# variáveis
x1 = S1.addVar(name="x1")
x2 = S1.addVar(name="x2")

# função objetivo
S1.setObjective(4 * x1 - x2, GRB.MAXIMIZE)

# restrições
S1.addConstr(7 * x1 - 2 * x2 <= 14)
S1.addConstr(x2 <= 3)
S1.addConstr(2 * x1 - 2 * x2 <= 3)
S1.addConstr(x1 <= 2) # nova restrição

# resolvendo o modelo
S1.optimize()

# imprimindo as solução
if S1.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % np.round(S1.objVal,1))
    for v in S1.getVars():
        print('%s = %g' % (v.varName, np.round(v.x,1)))
elif S1.status == GRB.INF_OR_UNBD:
    print('Modelo é inviável ou ilimitado')
else:
    print('Otimização finalizou com status %d' % S1.status)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xbfd26051
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+00   0.000000e+00   0.000000e+00      0s
       0    7.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  7.500000000e+00
Valor ótimo: 7.5
x1 = 2
x2 = 0.5


## Ao resolve $S_{1}$ temos que $\overline{x}^{1} = (2, 0.5)$ e $\overline{z}^1 = 7.5$.

## A variável $x_2 = 0.5$ não é inteira.

## Crie dois novos problemas $S_{11}$ e $S_{12}$, onde 
$S_{11} = S_1 \cup \{ x_2 \leq 0 \}$ e 
$S_{12} = S_1 \cup \{ x_2 \geq 1 \}$.

## Adicione $S_{11}$ e $S_{12}$ a lista de vértices ativos: 
$L = \{ S_2, S_{11}, S_{12} \}$

## Retire $S_2$ de $L$ para ser resolvido, $L = \{ S_{11}, S_{12} \}$ 

In [7]:
# modelo
S2 = gb.Model("S2")

# variáveis
x1 = S2.addVar(name="x1")
x2 = S2.addVar(name="x2")

# função objetivo
S2.setObjective(4 * x1 - x2, GRB.MAXIMIZE)

# restrições
S2.addConstr(7 * x1 - 2 * x2 <= 14)
S2.addConstr(x2 <= 3)
S2.addConstr(2 * x1 - 2 * x2 <= 3)
S2.addConstr(x1 >= 3)

# resolvendo o modelo
S2.optimize()

# imprimindo as solução
if S2.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % S2.objVal)
    for v in S2.getVars():
        print('%s = %g' % (v.varName, v.x))
elif S2.status == GRB.INF_OR_UNBD:
    print('Modelo é inviável ou ilimitado')
else:
    print('Otimização finalizou com status %d' % S2.status)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xaf020f90
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible or unbounded model
Modelo é inviável ou ilimitado


## O problema $S_2$ é inviável ou ilimitado.

## O ramo de $S_2$ é eliminado por inviabilidade ou por ser ilimitado.

## Retire $S_{12}$ de $L$ e resolva $S_{12}$, $L = \{ S_{11} \}$. 

In [8]:
# modelo
S12 = gb.Model("S12")

# variáveis
x1 = S12.addVar(name="x1")
x2 = S12.addVar(name="x2")

# função objetivo
S12.setObjective(4 * x1 - x2, GRB.MAXIMIZE)

# restrições
S12.addConstr(7 * x1 - 2 * x2 <= 14)
S12.addConstr(x2 <= 3)
S12.addConstr(2 * x1 - 2 * x2 <= 3)
S12.addConstr(x1 <= 2) # restrição adicionada em S_1
S12.addConstr(x2 >= 1) # nova restrição adicionada

# resolvendo
S12.optimize()

# imprimindo solução
if S12.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % S12.objVal)
    for v in S12.getVars():
        print('%s = %g' % (v.varName, v.x))
elif S2.status == GRB.INF_OR_UNBD:
    print('Modelo é inviável ou ilimitado')
else:
    print('Otimização finalizou com status %d' % S12.status)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0xd7015d77
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 5 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.000000000e+00
Valor ótimo: 7
x1 = 2
x2 = 1


## O problema $S_{12}$ possui solução inteira $\overline{x}^{12} = (2,1)$ e 
$\overline{z}^{12} = 7$.

## Defina $\underline{z} = 7$.

## O vértice $S_{12}$ é eliminado por otimalidade, pois $\overline{z}^{12} = \underline{z}^{12} = 7$

## Retire $S_{11}$ de $L$ e resolva $S_{11}$, $L = \{ \}$. 

In [9]:
# modelo
S11 = gb.Model("S11")

# variáveis
x1 = S11.addVar(name="x1")
x2 = S11.addVar(name="x2")

# função objetivo
S11.setObjective(4 * x1 - x2, GRB.MAXIMIZE)

# restrições
S11.addConstr(7 * x1 - 2 * x2 <= 14)
S11.addConstr(x2 <= 3)
S11.addConstr(2 * x1 - 2 * x2 <= 3)
S11.addConstr(x1 <= 2) # restrição de S_{1}
S11.addConstr(x2 <= 0) # nova restrição

# resolvendo o modelo
S11.optimize()

# imprimindo as solução
if S11.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % S11.objVal)
    for v in S11.getVars():
        print('%s = %g' % (v.varName, v.x))
elif S11.status == GRB.INF_OR_UNBD:
    print('Modelo é inviável ou ilimitado')
else:
    print('Otimização finalizou com status %d' % S2.status)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0x6c92db9c
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]
Presolve removed 5 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  6.000000000e+00
Valor ótimo: 6
x1 = 1.5
x2 = 0


## O problema $S_{11}$ possui valor ótimo, $\overline{z}^{11} = 6$, menor que a melhor solução inteira encontrada até agora, $\underline{z}=7$.

## Elimine o ramo de $S_{11}$ por limite.

## A lista $L$ está vazia, assim o problema foi resolvido.

## Solução ótima do MIP: $z^{*} = 7$ e $x^{*} = (2,1)$.